# 5.3.1 Layers without Parameters

In [9]:
import torch 
from torch import nn
from torch.nn import functional as F

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, X):
        return X-X.mean()
    
net = nn.Sequential(nn.Linear(8,128), CenteredLayer())
import torch 
from torch import nn
from torch.nn import functional as F

In [10]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, X):
        return X-X.mean()
    
net = nn.Sequential(nn.Linear(8,128), CenteredLayer())

# 5.3.2 Layers with Parameters

In [11]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self,X):
        linear = torch.matmul(X, self.weight.data)+self.bias.data
        return F.relu(linear)
linear = MyLinear(5,3)
linear.weight

Parameter containing:
tensor([[ 1.2804, -0.8459, -1.0987],
        [-1.2706,  1.2450, -0.9302],
        [ 0.5308,  0.2114, -1.4672],
        [-1.0717, -1.8174,  0.6573],
        [-0.2910, -0.4409, -0.4740]], requires_grad=True)

In [17]:
net = nn.Sequential(MyLinear(64,8), MyLinear(8,1))
net(torch.rand(2,64))

tensor([[0.],
        [0.]])

# Exercises

In [18]:
# 1. Design a layer that takes an input and computes a tensor reduction
class ex1_Layer(nn.Module):
    def __init__(self, in_units, units, ):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
    def forward(self,X):
        linear = torch.matmul(X, self.weight.data)
        return F.relu(linear)
net = nn.Sequential(ex1_Layer(20,4), ex1_Layer(4,1))
net(torch.rand(2,20))

tensor([[0.7670],
        [0.4206]])

In [14]:
# 2. Design a layer that returns the leading half of the Fourier coefficients of the data.

In [16]:
import numpy as np
def fourier_series_coeff_numpy(f, T, N, return_complex=False):
    """Calculates the first 2*N+1 Fourier series coeff. of a periodic function.
    Given a periodic, function f(t) with period T, this function returns the
    coefficients a0, {a1,a2,...},{b1,b2,...} such that:
    f(t) ~= a0/2+ sum_{k=1}^{N} ( a_k*cos(2*pi*k*t/T) + b_k*sin(2*pi*k*t/T) )
    If return_complex is set to True, it returns instead the coefficients
    {c0,c1,c2,...}
    such that:
    f(t) ~= sum_{k=-N}^{N} c_k * exp(i*2*pi*k*t/T)
    where we define c_{-n} = complex_conjugate(c_{n})

    Refer to wikipedia for the relation between the real-valued and complex
    valued coeffs at http://en.wikipedia.org/wiki/Fourier_series.

    Parameters
    ----------
    f : the periodic function, a callable like f(t)
    T : the period of the function f, so that f(0)==f(T)
    N_max : the function will return the first N_max + 1 Fourier coeff.
    Returns
    -------
    if return_complex == False, the function returns:
    a0 : float
    a,b : numpy float arrays describing respectively the cosine and sine coeff.
    if return_complex == True, the function returns:
    c : numpy 1-dimensional complex-valued array of size N+1
    """
    # From Shanon theoreom we must use a sampling freq. larger than the maximum
    # frequency you want to catch in the signal.
    f_sample = 2 * N
    # we also need to use an integer sampling frequency, or the
    # points will not be equispaced between 0 and 1. We then add +2 to f_sample
    t, dt = np.linspace(0, T, f_sample + 2, endpoint=False, retstep=True)
    y = np.fft.rfft(f(t)) / t.size
    if return_complex:
        return y
    else:
        y *= 2
        return y[0].real, y[1:-1].real, -y[1:-1].imag
    
class ex2_Layer(nn.Module):
    def __init__(self, in_units, units, ):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
    def forward(self,X):
        linear = fourier_series_coeff_numpy(f, T, N)
        return F.relu(linear)
